Tatoebaコーパスをダウンロードし、前処理するところを自分の手でやりたい場合は上から３番目のセルを実行する。スキップしたい場合は、上から３番目のセルだけ飛ばして実行する。

ただし、前処理済みのデータセットが既に用意されているので、必ずしも実行する必要はない。

In [1]:
import os

os.chdir("/")
!git clone https://github.com/hir777/Machine_Translation_Proto.git
!pip install sacremoses
!pip install spacy
!pip install tqdm
!pip install datasets
!pip install mecab-python3
!pip install unidic
!python3 -m unidic download

fatal: destination path 'Machine_Translation_Proto' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:26<00:00, 19.5MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.7/dist-packages/unidic/dicdir
Looking in indexes:

In [2]:
os.chdir("/Machine_Translation_Proto/")
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ここまで実行できたら、一度ランタイムを再起動する。
その際、上の二つのセルを忘れずに再実行する。

In [3]:
os.chdir("/Machine_Translation_Proto/corpus/src")

!python3 create_dataset.py \
--repo_path "/Machine_Translation_Proto" \
--len_filter \
--overlap_filter \
--ratio_filter \
--freq_filter

/Machine_Translation_Proto/corpus/data
Using custom data configuration en-ja-lang1=en,lang2=ja
Dataset tatoeba downloaded and prepared to /root/.cache/huggingface/datasets/tatoeba/en-ja-lang1=en,lang2=ja/0.0.0/b3ea9c6bb2af47699c5fc0a155643f5a0da287c7095ea14824ee0a8afd74daf6. Subsequent calls will reuse this data.
100% 1/1 [00:00<00:00, 202.61it/s]
Creating json from Arrow format: 100% 21/21 [00:00<00:00, 31.87ba/s]

Reformating a json file downloaded from tatoeba...
100% 208866/208866 [00:00<00:00, 912374.61it/s]

Converting a json file into list...
100% 208866/208866 [00:00<00:00, 2015983.16it/s]

Tokenizing English sentences...
 55% 114026/208866 [00:27<00:21, 4345.35it/s]
 55% 114252/208866 [00:27<00:22, 4137.51it/s]


In [4]:
os.chdir("/Machine_Translation_Proto/")
!bash pre-process.sh REPO_PATH="/Machine_Translation_Proto"

+ for ARGUMENT in "$@"
++ cut -f1 -d=
++ echo REPO_PATH=/Machine_Translation_Proto
+ KEY=REPO_PATH
+ KEY_LENGTH=9
+ VALUE=/Machine_Translation_Proto
+ export REPO_PATH=/Machine_Translation_Proto
+ REPO_PATH=/Machine_Translation_Proto
+ TRAIN_EN=/Machine_Translation_Proto/corpus/data/train.en
+ TRAIN_JA=/Machine_Translation_Proto/corpus/data/train.ja
+ VALID_EN=/Machine_Translation_Proto/corpus/data/valid.en
+ VALID_JA=/Machine_Translation_Proto/corpus/data/valid.ja
+ TEST_EN=/Machine_Translation_Proto/corpus/data/test.en
+ TRAIN_SP=/Machine_Translation_Proto/src/train_sp.py
+ ENCODE=/Machine_Translation_Proto/src/encode.py
+ cat /Machine_Translation_Proto/corpus/data/train.en /Machine_Translation_Proto/corpus/data/train.ja
+ python /Machine_Translation_Proto/src/train_sp.py --input train.enja --prefix bpe --vocab_size 8000 --character_coverage 0.9995
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: train.enja
  input_format: 
  model_prefix: bpe
  

In [ ]:
!fairseq-train \
    /Machine_Translation_Proto/data-bin \
    --fp16 \
    --save-interval 20 \
    --log-interval 5 \
    --log-format simple \
    --max-epoch 100 \
    --update-freq 1 \
    --max-update 30000 \
    --max-tokens 4000 \
    --arch bart_base \
    --encoder-normalize-before \
    --decoder-normalize-before \
    --encoder-embed-dim 512 \
    --encoder-ffn-embed-dim 4096 \
    --encoder-attention-heads 8 \
    --encoder-layers 8 \
    --decoder-embed-dim 512 \
    --decoder-ffn-embed-dim 4096 \
    --decoder-attention-heads 8 \
    --decoder-layers 8 \
    --share-all-embeddings \
    --dropout 0.3 \
    --attention-dropout 0.0 \
    --activation-dropout 0.0 \
    --activation-fn gelu \
    --optimizer adam \
    --adam-betas '(0.9, 0.999)' \
    --lr 0.0015 \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 4000 \
    --warmup-init-lr 1e-07 \
    --clip-norm 1.0 \
    --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.3 \
    | tee train.log

2022-06-13 15:20:33 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='gelu', adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='bart_base', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=1.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='/Machine_Translation_Proto/data-bin', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=8, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=4096, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=8, decoder_layers_to_keep=None, decoder_learned_pos=True, decoder_normalize_before=True, decoder_output_dim=512, device_id=0, disable_validation=False, distributed_b

In [ ]:
!fairseq-interactive data-bin \
    --buffer-size 1024 \
    --batch-size 128 \
    --path checkpoints/checkpoint_last.pt \
    --beam 1 \
    --lenpen 0.6 \
    < test.en \
    | grep '^H' \
    | cut -f 3 \
    | python src/decode.py \
    | tee output.txt \
    | sacrebleu /Machine_Translation_Proto/corpus/data/test.ja

In [ ]:
!head output.txt /Machine_Translation_Proto/corpus/data/test.ja

In [ ]:
import os
os.chdir("/Machine_Translation_Proto/src/")
from translation import Translation
en = input('英文を入力 > ')

tl = Translation("en", "ja")
tl.load(checkpoint_dir="/Machine_Translation_Proto/checkpoints/",
           checkpoint_file="checkpoint_last.pt",
           data_name_or_path="/Machine_Translation_Proto/data-bin/",
           path_bpe_model="/Machine_Translation_Proto/bpe.model",
          )
ja = tl.translate(en, 3, 0.6)
print(ja)